# Fundamentals of Machine Learning - 2022
## Report 2 - Classifying with convnets
Pablo Chehade  
Última modificación: 25/09/2022


### Introducción

Hay que construir 2 modelos: uno con ML clásico y otro con NN. El problema es uno de clasificación de imágenes (supervisado)

¿Cómo se evalúa el error? Qué parámetros se usarán?

TENGO QUE PONERME A RELEER LAS CLASES DE ERROR (LA ÚLTIMA PARTE) Y DE K-MEANS. Lo voy a necesitar


### 

In [ ]:
#Importo librerías
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score

### Análisis Exploratorio de los Datos

#### Cargo los datos

In [ ]:
import pickle
archivo = "datasets/faces_dict.p"
data = pickle.load(open(archivo), "rb")

#### Ver datos
Hay class imbalance?

In [ ]:
data


### Separo entre train y test

No podría ser un problema si justo en la partición de test caen muchas imágenes de la misma persona? Si no lo hago aleatorio, no estoy usando sin querer info. del test?

Recomendación guía 3:
t. Think about whether it would be convenient to use the stratified sampling 

In [ ]:
from sklearn.model_selection import train_test_split
#Divido el dataset entre train y test. A partir de ahora no utilizaré test
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42) ### Fijamos 20% para test y 42 es el random state para poder reproducir los resultados.

#Guardo el test set en un archivo pickle
test_set.to_pickle("datasets/data_test.pkl")

#Divido el train_set entre train_predictors y train_target
train_predictors = train_set.drop("dosel_forestal", axis=1)
train_target = train_set["dosel_forestal"].copy()

### Data augmentation?

Data augmentation artificially increases the size of the training set by generating
many realistic variants of each training instance. This reduces overfitting, making this
a regularization technique. The generated instances should be as realistic as possible: ideally, given an image from the augmented training set, a human should not be able
to tell whether it was augmented or not. Moreover, simply adding white noise will not
help; the modifications should be learnable (white noise is not).

Se explica bien en
https://neptune.ai/blog/data-augmentation-in-python
Existen librerías en Python para hacer este procedimiento casi automáticamente. Están todas explicadas en ese link

Podría usar una IA para rotar caras?

### MODELO 1: Random Forest

¿Por qué este modelo en particular? Corre rápido
A priori, el modelo va a ser extremadamente malo debido a la pequeña cantidad de datos y a la gran cantidad de dimensiones (64x64?)

RECOMENDACIÓN: use K-Means as a dimensionality reduction tool and train a classifier on the
reduced set (i.e., using the distances to the centroids).

In [ ]:
#### K-MEANS?

#### Modelo simple inicial

Creo un modelo con los parámetros por default salvo (...). Por default vale (...) e implica rápidamente overfitting

* En el práctico anterior: Con los parámetros por default el modelo tiene error nulo (accuracy de 100%) está overfitteando. Cambiando el parámetro max_depth a 10, el modelo tiene un error asociado y por lo tanto no overfittea (al menos no tanto como antes). Por default la variable max_depth está en None, es decir, los nodos se expanden hasta que las leaves sean puras, de ahí el overfitting quizás.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#Creo el modelo
RFmodel = RandomForestClassifier(n_jobs = -1, random_state = 42, n_estimators = 100, max_depth = 10, min_samples_split = 2, min_samples_leaf = 1)
#Lo entreno
RFmodel.fit(train_predictors, train_target)

##### Cross Validation

Calculo el error de validación del modelo. Uso cv=5 para que el programa corra rápido

#### Grid Search

En el práctico anterior se discutió este modelo con detalle, junto a los parámetros que se pueden variar. En este caso se decidió variar:


In [ ]:
#Tengo que poner que me devuelva el error de test para cada entrenamiento del modelo

##### ¿Cómo depende el error de train y el error de validación con la complejidad del modelo?

Para evaluar esto es necesario definir la "complejidad del modelo"

#### Feature Importances

En el caso de Desicion Trees o Random Forest, existe una métrica los features que determina cuáles son los más importantes en definir una categoría en el caso de clasificación

Intentar implementar permutation_importance.

En el práctico anterior se intentó emplear permutation_importance pero no se pudo utilizar debido al gran tiempo de ejecución. En función de esto, se decidió eliminar las variables con feature importancia menor a un threshold y entrenar el modelo con los mejores parámetros encontrados anteriormente.

#### Best Random Forest model

### MODELO 2: DL

#### Modelo simple inicial

##### Cross Validation

#### Grid Search

### Evaluación de test y exportación del modelo